In [67]:
from IPython.display import HTML

import math
import pandas as pd
import numpy as np

#from scipy.optimize import minimize
from scipy.optimize import differential_evolution    

#from lmfit import Minimizer, Parameters, report_fit

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import LinearAxis, Range1d
from bokeh.models import DatetimeTickFormatter, MonthsTicker, NumeralTickFormatter, Legend


output_notebook()

Loading BokehJS ...

In [68]:
import sys
sys.path.insert(1,'../covid-website')

import SISV as s
from data import Data
from ContactRate import contact_rate
from maxlik import fit_model, loglik_leastsquare , loglik_poisson, loglik_negbin
from maxlik import piecewiseexp_diffevol

In [69]:
source = 'Johns Hopkins'    
region = 'Europe'
state = 'Belgium'

cutoff_positive = 1
cutoff_death = 1
truncate = 0


d = Data(source=source, region=region, state=state, county="", cutoff_positive=cutoff_positive, cutoff_death=cutoff_death, truncate=truncate) 


In [70]:
#---------------------------------------------------------------------------------
#transform auxiliary values (which are bounded [0,1] into the breakpoint dates)
def aux_to_breakpoints(aux, x0, xn, minwindow):
    breakpoints = []
    bi1 = x0
    n=len(aux)
    for i,a in enumerate(aux):
        bi = a * (xn-(n-i)*minwindow - (bi1+minwindow)) + bi1+minwindow
        breakpoints.append(bi)
        bi1=bi   
    return breakpoints

#---------------------------------------------------------------------------------
def breakpoints_to_aux(breakpoints, x0, xn, minwindow):
    aux = []
    bi1 = x0
    n=len(breakpoints)
    for i,b in enumerate(breakpoints):
        temp = xn - (n-i)*minwindow - (bi1+minwindow)
        a = 1.0 if temp==0 else (b - bi1 - minwindow) / temp
        aux.append(a)
        bi1=b
    return aux  
    
#aux_to_breakpoints([0.5, 0.5], 0, 100, 10)


#---------------------------------------------------------------------------------


def override(key, overrides, default):
    return default if key not in overrides else overrides[key]

In [71]:
b = aux_to_breakpoints([0.5,0.5,0.5], 5, 100, 10)
a = breakpoints_to_aux(b, 5, 100, 10)
print(b)
print(a)

[42.5, 66.25, 83.125]
[0.5, 0.5, 0.5]


In [72]:
def merge_params(aux, params):
    
    segments = params['segments']
    minwindow = params['minwindow']
    
    params['i0'] = aux[0]
    
    params['beta0'] = aux[1]
    for i in range(1, segments+1):
        params['beta{}'.format(i)] = aux[i+1]
        
    breakpoints = aux_to_breakpoints(aux[-segments:], x[0], x[-1], minwindow) 
    for i, t in enumerate(breakpoints):
        params['t{}'.format(i+1)] = t

    return params


In [73]:
def diffevol_inner(aux, constants):
    #aux are the values of beta0, ..betan, t1, ..., tn

    x = constants['x']
    dy = constants['dy']
    params = constants['params']
    
    params = merge_params(aux, params)

    #calculate target
    
    yhat = s.SISV_J(x, params)  
    yprime = np.diff(yhat[:, s.cF])
    
    likelihood = loglik_leastsquare(dy, yprime, alpha=1) 
    
    #print(f"{likelihood:,}", aux, breakpoints)
    
    return likelihood

In [74]:
def SISV_diffevol(x, dy, params):
    
    segments = params['segments']  #0 means constant contact rate
    gamma = params['gamma']
    
    #prepare the bounds and guess vectors
    
    auxBounds = [(1,50000)]  #bounds for i0

    for i in range(segments+1): #beta's
        auxBounds.append((0.01*gamma, 6*gamma))
    
    for i in range(segments): #ti's
        auxBounds.append((0,1))
    
    constants = {'x':x, 'dy':dy, 'params':params}
    
    #find the optimal breakpoints
    
    res = differential_evolution(func=diffevol_inner, bounds=auxBounds, args=[constants], updating='immediate')
    #print(res)
    
    #do the regression for the optimum breakpoints
    return merge_params(res.x, params)





In [75]:
x = d.x[d.minD:]
y = d.fatalities[d.minD:]
dy = d.dfatalities

#params, breakpoints, likelihood, expfit = piecewiseexp_diffevol(x[1:], dy, breaks=4, minwindow=7)
#print(breakpoints)
#print(params)





In [76]:
def print_params(title, params):
    print('----------')
    print(title)
    print("detect:", params["detection_rate"])
    print("i0:", params["i0"])
    print("R0:", params['beta0']/params['gamma'])
    for i in range(1, 1+params['segments']):
          print(params['t{}'.format(i)],":", params['beta{}'.format(i)]/params['gamma'])


In [77]:
params1 = {
    "population"    : d.population,
    'exp_stages'    : 1,
    'inf_stages'    : 1,
    'crit_stages'   : 1,
    'test_stages'   : 1,
    "death_rate"    : 0.73*1e-2,
    "detection_rate": 10e-2,
    "gamma_exp"     : 1/4,
    "gamma"         : 1/4,
    "gamma_crit"    : 1/14,
    "gamma_pos"     : 1/14,
    "immun"         : 0,
    "vacc_start"    : 365,
    "vacc_rate"     : 0.0/365,  #30% of population per year
    "vacc_immun"    : 1/180,
    
    "interv"        : 'piecewise constant',
    "init_beta"     : '',
    'segments'      : 5,
    'minwindow'     : 7,
}



In [78]:
%%time

params = SISV_diffevol(x, dy, params1)



CPU times: user 12min 57s, sys: 356 ms, total: 12min 58s
Wall time: 13min 14s


In [82]:
y1 = s.SISV_J(d.x, params) 

def print_params(title, params):
    print('----------')
    print(title)
    print("detect:", params["detection_rate"])
    print("i0:", params["i0"])
    print("R0:", params['beta0']/params['gamma'])
    for i in range(1, 1+params['segments']):
          print(params['t{}'.format(i)],":", params['beta{}'.format(i)]/params['gamma'])

print_params("p1", params)

p = figure(title='Excess Fatalities', plot_width=800, plot_height=600 , y_axis_type="log")
p.y_range.start = 1

#historical data
r0 = p.line(d.xd[d.minD+1:], d.dfatalities, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1 = p.circle(d.xd[d.minD+1:], d.dfatalities, size=5, color="red", alpha=0.3)

#plot 7-day rolling average
rolling = pd.DataFrame(data = d.dfatalities).interpolate().rolling(7).mean()
r2 = p.line(d.xd[d.minD+1:], rolling.loc[:,0].values, line_width=1, line_color='red')

#fit
r3_1 = p.line(d.xd[d.minD+1:], np.diff(y1[d.minD:,s.cF]), line_width=1, line_color='black', line_dash='solid', alpha=0.7)

#-------------

r0_p = p.line(d.xd[d.minP+1:], d.dpositives, line_width=1, line_color='red', line_dash='dotted', alpha=0.3)
r1_p = p.circle(d.xd[d.minP+1:], d.dpositives, size=5, color="red", alpha=0.3)

r3_1_p = p.line(d.xd[d.minP+1:], np.diff(y1[d.minP:,s.cP]), line_width=1, line_color='black', line_dash='solid', alpha=0.7)

legend = [
    ("COVID fatalities"   , [r0, r1]),
    ("COVID 7-day average"   , [r2]),
    
    ("p1", [r3_1]),

    ("COVID positives"   , [r0_p, r1_p]),
    ("p1_p", [r3_1_p]),
]

    
p.add_layout(Legend(items=legend, location='center'), 'right')
   
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

p.xaxis.ticker = MonthsTicker(months=list(range(1,13)))

p.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

p.xgrid.ticker = p.xaxis.ticker

p.xaxis.major_label_orientation = math.pi/4


p.extra_x_ranges['x2'] = Range1d(d.x[0], d.x[-1])
ax2 = LinearAxis(x_range_name="x2", axis_label="days")
p.add_layout(ax2, 'below')


p.ygrid.grid_line_color = 'navy'
p.ygrid.grid_line_alpha = 0.3
p.ygrid.minor_grid_line_color = 'navy'
p.ygrid.minor_grid_line_alpha = 0.1

p.yaxis[0].axis_label = 'Number of deaths per day'

#p.toolbar_location = None
show(p)

rplot = figure(title='R0')
R0_1 = contact_rate(d.x, p1) / p1['gamma']
rplot.line(d.x, R0_1, line_width=1, line_color='black', line_dash='solid', alpha=1)
show(rplot)


----------
p1
detect: 0.1
i0: 15502.908659420009
R0: 2.4532291880692765
32.46237838771417 : 0.07459700103490728
78.40066700077973 : 1.5488156431343518
224.36302663658557 : 3.6254702560732674
236.99985529993901 : 0.7181739522376444
258.93683730852695 : 1.1571546438255873


NameError: name 'p1' is not defined